In [ ]:
import sqlite3
import textwrap

import sqlparse
from IPython.core.magic import register_cell_magic
from tabulate import tabulate

conn = sqlite3.connect(':memory:')


def run_and_print(sql: str) -> None:
    try:
        curr = conn.execute(sql)
    except sqlite3.OperationalError as e:
        print("Error:\n", sql)
        raise e

    rows = curr.fetchall()
    if curr.description is None:
        return

    print(textwrap.indent(tabulate(rows, headers=[d[0] for d in curr.description]), '  '), end='\n\n')


@register_cell_magic
def sql(_line: str, cell: str) -> None:
    for sql in sqlparse.split(cell):
        print(sql)
        run_and_print(sql)

@register_cell_magic
def sql_fresh(_line: str, cell: str) -> None:
    global conn
    conn = sqlite3.connect(':memory:')
    conn.execute('PRAGMA recursive_triggers = OFF;')
    sql(_line, cell)

In [ ]:
%%sql_fresh


-- 1. Create the table
CREATE TABLE todo_list (
    ordr INTEGER,
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    todo_txt TEXT
);

-- 2. Insert dummy data
INSERT INTO todo_list (ordr, todo_txt) VALUES (1, 'Buy groceries');
INSERT INTO todo_list (ordr, todo_txt) VALUES (2, 'Finish the report');
INSERT INTO todo_list (ordr, todo_txt) VALUES (3, 'Call Alice to schedule a meeting');
INSERT INTO todo_list (ordr, todo_txt) VALUES (4, 'Book flight tickets');
INSERT INTO todo_list (ordr, todo_txt) VALUES (5, 'Renew gym membership');
INSERT INTO todo_list (ordr, todo_txt) VALUES (6, 'Prepare presentation slides');
INSERT INTO todo_list (ordr, todo_txt) VALUES (7, 'Clean the house');
INSERT INTO todo_list (ordr, todo_txt) VALUES (8, 'Pay utility bills');
INSERT INTO todo_list (ordr, todo_txt) VALUES (9, 'Read chapter 5 of the book');
INSERT INTO todo_list (ordr, todo_txt) VALUES (10, 'Update resume');
INSERT INTO todo_list (ordr, todo_txt) VALUES (11, 'Attend yoga class');
INSERT INTO todo_list (ordr, todo_txt) VALUES (12, 'Fix the leaking faucet');
INSERT INTO todo_list (ordr, todo_txt) VALUES (13, 'Plan weekend trip');
INSERT INTO todo_list (ordr, todo_txt) VALUES (14, 'Organize office desk');
INSERT INTO todo_list (ordr, todo_txt) VALUES (15, 'Submit tax documents');

-- 3. Create triggers
-- 3.1 Trigger for updating ordr
CREATE TRIGGER reorder_todo_list_after_insert_update
AFTER UPDATE OF ordr ON todo_list
BEGIN
    UPDATE todo_list
    SET ordr = ordr + (
        CASE
            WHEN NEW.ordr == OLD.ordr THEN 1  -- Piggybacked insert
            WHEN NEW.ordr < OLD.ordr THEN 1  -- Moving up
            WHEN NEW.ordr > OLD.ordr THEN -1 -- Moving down
        END
    )
    WHERE
        id != NEW.id
        AND
        (
            (NEW.ordr != OLD.ordr and ordr BETWEEN MIN(NEW.ordr, OLD.ordr) AND MAX(NEW.ordr, OLD.ordr)) -- Moving up or down
            OR
            (NEW.ordr == OLD.ordr AND  ordr >= NEW.ordr) -- Piggybacked insert
        )
    ;
END;

-- 3.2 Trigger for inserting a new task
CREATE TRIGGER reorder_todo_list_after_insert
AFTER INSERT ON todo_list
FOR EACH ROW
BEGIN
    -- Shift existing tasks to make room for the new task
    UPDATE todo_list --piggypack on the update trigger
    SET ordr = NEW.ordr
        WHERE id == NEW.id;
    -- SET ordr = ordr + 1
    -- WHERE ordr >= NEW.ordr
    --   AND id != NEW.id;
END;

-- 3.3 Trigger for deleting a task
CREATE TRIGGER reorder_todo_list_after_delete
AFTER DELETE ON todo_list
FOR EACH ROW
BEGIN
    UPDATE todo_list
    SET ordr = ordr - 1
    WHERE ordr > OLD.ordr;
END;


-- Original Table
SELECT * FROM todo_list
ORDER BY ordr ASC;

-- Tests

-- A. Move task 15 to position 6
-- UPDATE todo_list
-- SET ordr = 6
-- WHERE id = 15;

-- SELECT * FROM todo_list ORDER BY ordr ASC;

-- B. Delete task 8
-- DELETE FROM todo_list
-- WHERE id = 7;
-- SELECT * FROM todo_list ORDER BY ordr ASC;

-- C. Insert a new task at position 5
INSERT INTO todo_list (ordr, todo_txt) VALUES (5, 'Write blog post');
SELECT * FROM todo_list ORDER BY ordr ASC;